In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

# Get the current working directory
notebook_dir = os.getcwd()

# Change the working directory to the root directory of the project
os.chdir("../")

# move up to the parent directory(project root)
sys.path.append(os.path.abspath(os.path.join(notebook_dir, "..")))

os.getcwd()

'c:\\Users\\mhannani\\Documents\\sentiment_analysis'

In [3]:
import torch
from pathlib import Path
from pandas import read_csv
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from pandas import read_csv
import numpy as np


from src.data.sentiment_data import SentimentDataset
from src.data.split import DataSplitter
from src.utils.parsers import parse_toml
from src.utils.get import get_model_tokenizer
from src.data.split import DataSplitter
from src.utils.get import get_model_tokenizer
from src.models.classifier import SentimentClassifier

c:\Users\mhannani\Documents\sentiment_analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def compute_metrics(p):
    print("p[]", p)
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    print(labels, pred)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [5]:
# configration filepath
CONFIG_FILE = Path("configs/config.toml")

# read configuration object
config = parse_toml(CONFIG_FILE)

# useful variables
data_root = Path(config['data']['root'])
processed_data = config['data']['processed']
preprocessed_mac_csv_filename = config['data']['preprocessed_mac_csv']

# batch size
batch_size = int(config['params']['batch_size'])

# preprocessed csv file
preprocessed_mac_csv = data_root / processed_data / preprocessed_mac_csv_filename

# data splitter
data_splitter = DataSplitter(config, preprocessed_mac_csv)

# split data into train and val sets
train_df, val_df = data_splitter.split()

In [38]:
"""Train custom BERT model"""

# list of models to train
model_id_to_train = "google-bert/bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_id_to_train)
model = AutoModelForSequenceClassification.from_pretrained(model_id_to_train, num_labels=3)

model
# get the model and the tokenizer
# tokenizer, model = get_model_tokenizer(model_id_to_train)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [39]:
from src.utils.model import get_model_trainable_layers


print("get_model_trainable_layers(model): ", len(get_model_trainable_layers(model)))


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [24]:
model.config.num_labels

3

In [42]:
model = SentimentClassifier(config, model, True)

In [44]:
print("get_model_trainable_layers(model): ", get_model_trainable_layers(model))

SentimentClassifier(
  (bert): SentimentClassifier(
    (bert): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSe

In [32]:
import datetime
# train dataset
train_data = SentimentDataset(train_df, tokenizer)

# valid dataset
eval_data = SentimentDataset(val_df, tokenizer)

model_id_to_train = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_id_to_train)
model = AutoModelForSequenceClassification.from_pretrained(model_id_to_train, num_labels=3)

model = SentimentClassifier(config, model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
model

SentimentClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features

In [35]:
print("get_model_trainable_layers(model): ", len(get_model_trainable_layers(model)))

SentimentClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features

In [27]:
model.config.num_labels

3

In [28]:
model.config

BertConfig {
  "_name_or_path": "google-bert/bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

In [29]:

# training args
training_args = TrainingArguments(
    output_dir=f"output_bert_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}/{model_id_to_train}",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    seed=0,
    save_strategy = "epoch"
)

# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset = eval_data,
)

# train current model
trainer.train()

# evaluate current model
evaluation_results = trainer.evaluate()

print(evaluation_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                             
100%|██████████| 3/3 [00:13<00:00,  4.63s/it]


{'train_runtime': 13.8842, 'train_samples_per_second': 0.432, 'train_steps_per_second': 0.216, 'train_loss': 1.1197638511657715, 'epoch': 1.0}


c:\Users\mhannani\Documents\sentiment_analysis\venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mhannani\Documents\sentiment_analysis\venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 3/3 [00:08<00:00,  2.79s/it]

p[] <transformers.trainer_utils.EvalPrediction object at 0x000001B7A4E489D0>
[1 2 2 2 2 2] [0 0 0 1 1 1]
{'eval_loss': 1.1571760177612305, 'eval_accuracy': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 12.8324, 'eval_samples_per_second': 0.468, 'eval_steps_per_second': 0.234, 'epoch': 1.0}


In [83]:
# Example random input data
input_ids = torch.randint(0, 10000, (3, 20))  # Random input IDs with shape (batch_size, sequence_length)
attention_mask = torch.randint(0, 2, (3, 20))  # Random attention mask with shape (batch_size, sequence_length)
# Mock labels
labels = torch.randint(0, 3, (3,))  # Random labels with shape (batch_size,)


model_id_mapping = {
    "bert-base-multilingual-cased": 'google-bert/bert-base-multilingual-cased',
    "bert-base-arabic": "asafaya/bert-base-arabic",
    "darijabert-arabizi": "SI2M-Lab/DarijaBERT-arabizi",
    "DarijaBERT": "SI2M-Lab/DarijaBERT",
    "bert-base-arabertv2": "aubmindlab/bert-base-arabertv2",
}

for key, value in model_id_mapping.items():
    print(f"============== {value} ==================")
    _, model = get_model_tokenizer(key)
    
    print("model type: ", type(model))
    model = SentimentClassifier(config, model)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    # print(outputs)
    for name, param in model.named_parameters():
        
        if param.requires_grad:
            print(f"Layer: {name}, Trainable: {param.requires_grad}")
    print("=================================================")
    

============== google-bert/bert-base-multilingual-cased ==================


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model type:  <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
Layer: bert.classifier.classifier.0.weight, Trainable: True
Layer: bert.classifier.classifier.0.bias, Trainable: True
Layer: bert.classifier.classifier.3.weight, Trainable: True
Layer: bert.classifier.classifier.3.bias, Trainable: True
============== asafaya/bert-base-arabic ==================


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model type:  <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
Layer: bert.classifier.classifier.0.weight, Trainable: True
Layer: bert.classifier.classifier.0.bias, Trainable: True
Layer: bert.classifier.classifier.3.weight, Trainable: True
Layer: bert.classifier.classifier.3.bias, Trainable: True
============== SI2M-Lab/DarijaBERT-arabizi ==================


...

model type:  <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
Layer: bert.classifier.classifier.0.weight, Trainable: True
Layer: bert.classifier.classifier.0.bias, Trainable: True
Layer: bert.classifier.classifier.3.weight, Trainable: True
Layer: bert.classifier.classifier.3.bias, Trainable: True
============== SI2M-Lab/DarijaBERT ==================


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model type:  <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
Layer: bert.classifier.classifier.0.weight, Trainable: True
Layer: bert.classifier.classifier.0.bias, Trainable: True
Layer: bert.classifier.classifier.3.weight, Trainable: True
Layer: bert.classifier.classifier.3.bias, Trainable: True
============== aubmindlab/bert-base-arabertv2 ==================


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model type:  <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
Layer: bert.classifier.classifier.0.weight, Trainable: True
Layer: bert.classifier.classifier.0.bias, Trainable: True
Layer: bert.classifier.classifier.3.weight, Trainable: True
Layer: bert.classifier.classifier.3.bias, Trainable: True


In [ ]:
model